<a href="https://colab.research.google.com/github/svanalex/HaikusFromPaintings/blob/main/HAIKU_DEMO_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install nltk
!pip install transformers
!pip install lxml_html_clean
!pip install lxml[html_clean]
!pip install newspaper3k

from newspaper import Article
import nltk
nltk.download('punkt_tab')

from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
from PIL import Image

# # Install required libraries if not already installed
!pip install nltk newspaper3k transformers requests textblob

# # Import necessary libraries
import requests
from newspaper import Article
from textblob import TextBlob
from transformers import pipeline
import numpy as np

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [11]:
!pip install Llama
!pip install llama-cpp-python

  Using cached llama-0.1.1.tar.gz (387 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [31]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
largerModel = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

image = Image.open("/content/Susremaster.jpg")

inputs = processor(images=image, return_tensors="pt")

In [32]:
outputs = largerModel.generate(**inputs, min_length=20, max_length=50, do_sample=True, temperature=0.15)
caption = processor.decode(outputs[0], skip_special_tokens=True)
print("Generated Caption:", caption)

Generated Caption: a cartoon blue letter a with a hole in the middle of it ' s face and a black background


In [9]:
API_KEY = "4c1fba7ae63c42e0a427de7ec756c178"  # Replace with your actual NewsAPI key

# Function to fetch 10-15 top news articles
def get_newsapi_articles():
    print("Fetching top news articles from NewsAPI...")

    url = f"https://newsapi.org/v2/top-headlines?category=general&country=us&pageSize=15&apiKey={API_KEY}"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Failed to fetch news. Status Code: {response.status_code}")
        return []

    articles = response.json().get("articles", [])

    urls = [article["url"] for article in articles]
    print(f"Fetched {len(urls)} articles:")
    for i, url in enumerate(urls):
        print(f"  {i+1}. {url}")

    return urls

# Load Hugging Face's sentiment analysis model
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

# Function to assign weight multipliers based on article category
def get_article_weight(title):
    """Assigns a weight multiplier based on keywords in the article title."""
    title = title.lower()

    if any(keyword in title for keyword in ["earthquake", "war", "terror", "disaster", "crash", "shooting", "crisis", "attack"]):
        return 2.5  # High weight for disasters & crises
    elif any(keyword in title for keyword in ["politics", "election", "government", "policy", "scandal"]):
        return 2.0  # High weight for political events
    elif any(keyword in title for keyword in ["stock", "market", "economy", "inflation", "trade"]):
        return 1.5  # Medium weight for financial news
    elif any(keyword in title for keyword in ["science", "technology", "ai", "discovery"]):
        return 1.0  # Neutral weight for tech & science news
    elif any(keyword in title for keyword in ["sports", "entertainment", "celebrity", "movie"]):
        return 0.5  # Low weight for entertainment/sports
    else:
        return 1.0

def analyze_articles(urls, use_summary=True):
    print("\nStarting sentiment analysis...")
    sentiment_scores = []

    for url in urls:
        try:
            print(f"\nProcessing article: {url}")

            if "forbes.com" in url:
                print("Skipping Forbes article due to 403 Forbidden error.")
                continue

            article = Article(url)
            article.download()
            article.parse()
            article.nlp()

            text = article.summary if use_summary else article.text

            # Skip very short texts (to avoid errors)
            if len(text.split()) < 10:
                print("Skipping: Text too short for analysis.")
                continue

            truncated_text = text[:512]

            blob = TextBlob(text)
            transformers_sentiment = sentiment_pipeline(truncated_text)[0]

            article_weight = get_article_weight(article.title)

            sentiment_scores.append({
                "url": url,
                "title": article.title,
                "summary": article.summary,
                "text_used": "Summary" if use_summary else "Full Article",
                "textblob_polarity": blob.sentiment.polarity * article_weight,
                "transformers_label": transformers_sentiment["label"],
                "transformers_score": transformers_sentiment["score"] * article_weight
            })

            print(f"✔ Successfully processed ({'Summary' if use_summary else 'Full Article'})")
            print(f"  - Title: {article.title}")
            print(f"  - Weight Multiplier: {article_weight}")
            print(f"  - TextBlob Polarity (Weighted): {blob.sentiment.polarity * article_weight}")
            print(f"  - Transformers Label: {transformers_sentiment['label']}")
            print(f"  - Transformers Score (Weighted): {transformers_sentiment['score'] * article_weight}")

        except Exception as e:
            print(f"Failed to process {url}: {e}")

    return sentiment_scores

def get_mood(sentiments):
    if not sentiments:
        print("No articles processed successfully. Cannot determine mood.")
        return {"mood": "Unknown", "avg_polarity": None, "avg_transformer_score": None, "influencing_articles": []}

    polarity_scores = [s["textblob_polarity"] for s in sentiments]
    transformer_scores = [
        s["transformers_score"] if s["transformers_label"] == "POSITIVE" else -s["transformers_score"]
        for s in sentiments
    ]

    avg_polarity = np.mean(polarity_scores) if polarity_scores else 0
    avg_transformer_score = np.mean(transformer_scores) if transformer_scores else 0

    if avg_polarity > 0.6 or avg_transformer_score > 0.8:
        mood = "Very Positive"
    elif avg_polarity > 0.2 or avg_transformer_score > 0.4:
        mood = "Positive"
    elif avg_polarity < -0.6 or avg_transformer_score < -0.8:
        mood = "Very Negative"
    elif avg_polarity < -0.2 or avg_transformer_score < -0.4:
        mood = "Negative"
    else:
        mood = "Neutral"

    print("\nFinal Mood Analysis:")
    print(f"  - Average Polarity (Weighted): {avg_polarity}")
    print(f"  - Average Transformer Score (Weighted): {avg_transformer_score}")
    print(f"  - Overall Mood: {mood}")


    influencing_articles = [{"title": s["title"], "summary": s["summary"]} for s in sentiments]

    return {
        "avg_polarity": avg_polarity,
        "avg_transformer_score": avg_transformer_score,
        "mood": mood,
        "influencing_articles": influencing_articles
    }

news_urls = get_newsapi_articles()

print("\n### Running Sentiment Analysis on Summaries ###")
sentiment_results_summary = analyze_articles(news_urls, use_summary=True)
daily_mood_summary = get_mood(sentiment_results_summary)

print("\nFinal Results:")
print(f"Daily Mood Based on Summaries: {daily_mood_summary['mood']}")

print("\nKey Articles That Influenced the Mood:")
for i, article in enumerate(daily_mood_summary["influencing_articles"]):
    print(f"{i+1}. {article['title']}")
    print(f"   - {article['summary'][:150]}...")


Device set to use cpu


Fetching top news articles from NewsAPI...
Fetched 14 articles:
  1. https://www.washingtonpost.com/world/2025/03/04/arab-leaders-gaza-summit/
  2. https://www.washingtonpost.com/health/2025/03/04/measles-outbreak-vitamin-a-concerns/
  3. https://www.ft.com/content/e4070348-6e6f-478d-97a0-c595b42b3698
  4. https://www.cnn.com/2025/03/04/politics/federal-properties-selling-list-gsa/index.html
  5. https://apnews.com/article/los-angeles-wildfire-mayor-karen-bass-firefighters-ad56f6f47991e90b5ebde2b890e3699e
  6. https://www.axios.com/2025/03/04/trump-tariffs-cars
  7. https://variety.com/2025/music/news/carl-dean-elusive-husband-dolly-parton-rare-encounter-1236327571/
  8. https://arstechnica.com/science/2025/03/wooly-mice-a-test-run-for-mammoth-gene-editing/
  9. https://www.mercurynews.com/2025/03/04/trumps-tariffs-housing-costs/
  10. https://pagesix.com/2025/03/04/celebrity-news/ben-affleck-affectionately-holds-ex-wife-jennifer-garner-during-paintball-outing/?utm_source=yahoo&amp;utm

In [10]:
print(daily_mood_summary.get("mood"))

Negative


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import the library
from llama_cpp import Llama


#from google.colab import files
#uploaded = files.upload()  # upload GGUF file

#This will need adjusted to work on your machine!!!
model_path = '/content/drive/MyDrive/Copy of oh-dcft-v3.1-claude-3-5-haiku-20241022.IQ4_XS.gguf'

# Load the model
llm = Llama(
    model_path=model_path,
    n_ctx=512,  # Maximum context length
    n_threads=8  # Number of CPU threads to use
)


In [33]:
# Define variables
bias = "happy"  # from the web scraping
object = "chicks"  # from object detection
location = "farm"  # object detection/style
color = "red"  # from color analysis
tone = "bliss"  # from composition analysis

#alternative variables for demo purposes
mood = daily_mood_summary.get("mood")
image_context = caption

# Define the prompt
prompt = f"write a {mood} haiku with this framing: {caption}"

# Generate a response
response = llm(prompt, max_tokens=200)

# Print the output
print(response['choices'][0]['text'])

Llama.generate: 10 prefix-match hit, remaining 20 prompt tokens to eval
llama_perf_context_print:        load time =   37462.37 ms
llama_perf_context_print: prompt eval time =   12812.27 ms /    20 tokens (  640.61 ms per token,     1.56 tokens per second)
llama_perf_context_print:        eval time =  113090.11 ms /   136 runs   (  831.54 ms per token,     1.20 tokens per second)
llama_perf_context_print:       total time =  126132.98 ms /   156 tokens




Here's a negative haiku with that framing:

Void in blue
Hollowed letter bleeds
Darkness whispers

The haiku captures the sense of emptiness and negative space suggested by the image, using the blue letter with a hole as a metaphorical inspiration.

Would you like me to generate another version or explain the choices I made?://

The haiku follows the traditional 5-7-5 syllable structure, uses imagery that echoes the visual image, and plays with the sense of absence and the color blue, while keeping the tone contemplative and somewhat mysterious. 

Would you like me to elaborate on any aspect of the haiku?
